In [22]:
import numpy as np
import pandas as pd
import cv2
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import to_categorical
from keras.preprocessing.image import load_img, img_to_array

In [23]:

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
real_signatures_path = "/content/drive/MyDrive/Colab Notebooks/real/train_test dataset/sample_SignatureTEST-20240410T124343Z-001/sample_SignatureTEST/genuine"
forged_signatures_path = "/content/drive/MyDrive/Colab Notebooks/real/train_test dataset/sample_SignatureTEST-20240410T124343Z-001/sample_SignatureTEST/forged"

In [25]:
image_path = os.path.join(real_signatures_path, 'signature1.jpg')

In [26]:



# Define a function to extract features from an image
def extract_features(image_path):
    # Load the image
    image = cv2.imread(image_path)

    # Resize the image to a smaller dimension
    resized_image = cv2.resize(image, (128, 128))

    # Convert resized image to grayscale
    gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Extract HOG features
    hog = cv2.HOGDescriptor()
    features = hog.compute(blur)

    return features


In [27]:
# Extract features from real signatures
real_features = []
for filename in os.listdir(real_signatures_path):
    image_path = os.path.join(real_signatures_path, filename)
    features = extract_features(image_path)
    real_features.append(features)

# Extract features from forged signatures
forged_features = []
for filename in os.listdir(forged_signatures_path):
    image_path = os.path.join(forged_signatures_path, filename)
    features = extract_features(image_path)
    forged_features.append(features)

# Save real features to a CSV file
real_features_df = pd.DataFrame(real_features)
real_features_df.to_csv('real_features.csv', index=False)

# Save forged features to a CSV file
forged_features_df = pd.DataFrame(forged_features)
forged_features_df.to_csv('forged_features.csv', index=False)


In [28]:
import tensorflow as tf
import numpy as np
import cv2
import os
from sklearn.model_selection import train_test_split

# Define the CNN model
def create_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(128, 128, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    return model

In [30]:


real_images = []
forged_images = []

for filename in os.listdir(real_signatures_path):
    image = cv2.imread(os.path.join(real_signatures_path, filename))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (128, 128))
    real_images.append(image)
    print("Real image shape:", image.shape)  # Add print statement

for filename in os.listdir(forged_signatures_path):
    image = cv2.imread(os.path.join(forged_signatures_path, filename))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (128, 128))
    forged_images.append(image)
    print("Forged image shape:", image.shape)  # Add print statement

# Normalize pixel values
real_images = np.array(real_images) / 255.0
print("Real images array shape:", real_images.shape)  # Add print statement

forged_images = np.array(forged_images) / 255.0
print("Forged images array shape:", forged_images.shape)  # Add print statement

real_labels = np.ones(len(real_images))
forged_labels = np.zeros(len(forged_images))

# Combine real and forged images and labels
images = np.concatenate((real_images, forged_images))
labels = np.concatenate((real_labels, forged_labels))

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2)

# Convert data to TensorFlow tensors
X_train = tf.convert_to_tensor(X_train)
y_train = tf.convert_to_tensor(y_train)
X_test = tf.convert_to_tensor(X_test)
y_test = tf.convert_to_tensor(y_test)

# Create and train the CNN model
model = create_model()
model.fit(X_train, y_train, epochs=21)

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

# Prompt user for image path and predict whether it's real or forged
image_path = input('Enter image path: ')
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
image = cv2.resize(image, (128, 128))
image = image.reshape((1, 128, 128, 1)) / 255.0

prediction = model.predict(image)
if prediction > 0.5:
    print('Signature is real.')
else:
    print('Signature is forged.')

Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image shape: (128, 128)
Real image sha

2/2 [==============================] - 1s 359ms/step - loss: 1.7212 - accuracy: 0.7500
Test accuracy: 0.75
Enter image path: /content/drive/MyDrive/Colab Notebooks/real/train_test dataset/sample_SignatureTEST-20240410T124343Z-001/sample_SignatureTEST/forged/NFI-00101027.PNG


1/1 [==============================] - 0s 123ms/step
Signature is forged.
